In [1]:
# import all the required library
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string 
import numpy as np
import pandas as pd

# import the data
sample_data_path = "data/sample_data/"
jobs = pd.read_csv(sample_data_path + 'jobs_sample_cleared.csv')
apps = pd.read_csv(sample_data_path + 'apps_sample.csv')
users = pd.read_csv(sample_data_path + 'users_sample.csv')

In [2]:
# print out first few job sample data
jobs.head()

,Unnamed: 0,JobID,Title,Description,Requirements,State,City
0,0,18,Immediate Opening,Immediate Opening - AR Clerk is needed foward...,Please refer to the Job Description to view th...,FL,Orlando
1,1,40,Administrative Assistant,ADMINISTRATIVE ASSISTANT Part Time The West Or...,Please refer to the Job Description to view th...,FL,Orlando
2,2,370,Domestic Violence,Domestic Violence and Traumatic Brain Injury P...,Please refer to the Job Description to view th...,FL,Tallahassee
3,3,413,Maintenance Tech,Northwest Houston manufacturer is adding to th...,"For consideration, Candidate will possessAt le...",TX,Houston
4,4,450,Customer Service Rep,Customer Service Rep - Provide Customer Servi...,Please refer to the Job Description to view th...,FL,Coral Springs


In [3]:
# convert all the properties of each job into a string
jobs_collection = []
for i in range (len(jobs)):
    text = "{} {} {} {} {}".format(jobs.loc[i, 'Title'],
                                   jobs.loc[i, 'Description'],
                                   jobs.loc[i, 'Requirements'],
                                   jobs.loc[i, 'State'],
                                   jobs.loc[i, 'City'])
    jobs_collection.append(text)

In [4]:
# print the first job in jobs_collection
jobs_collection[0]

'Immediate Opening Immediate Opening - AR Clerk is needed  foward resumes to hr.wallsfurniture@aol.com hr.wallsfurniture@aol.com WEB OS69883Accounts Receivable/Cash Applications Clerk is needed,candidates Must have advanced knowledge of computers and peripheral office devices.For immediate consideration candidates should foward resumes to : hr.wallsfurniture@aol.com                                                  Source - Orlando Sentinel Please refer to the Job Description to view the requirements for this job FL Orlando'

In [5]:
# define a clean_string function that remove the punctuation, lower all the letter in the text
from nltk.corpus import stopwords
import re
stopwords = stopwords.words('english')
def clean_string(text):
    # remove punctuation
    text = ''.join([word for word in text if word not in string.punctuation])
    # lower all letters 
    text = text.lower()
    # remove stop words 
    text = ' '.join([word for word in text.split() if word not in stopwords])
    # remove any digit and letter combinations
    text = re.sub(r'\w*[0-9]\w*','',text)
    return text

In [6]:
# clean all the jobs string in jobs_collections
cleaned_jobs_collection = list(map(clean_string, jobs_collection))
cleaned_jobs_collection

['immediate opening immediate opening ar clerk needed foward resumes hrwallsfurnitureaolcom hrwallsfurnitureaolcom web  receivablecash applications clerk neededcandidates must advanced knowledge computers peripheral office devicesfor immediate consideration candidates foward resumes hrwallsfurnitureaolcom source orlando sentinel please refer job description view requirements job fl orlando',
 'administrative assistant administrative assistant part time west orange healthcare district part time administrative assistant position support executive director board trustees performing reception secretarial administrative financial functions needed must  years related experience proficient microsoft word excel power point working knowledge quickbooks plus send resume kennethharkerhealthcentralorg eoe source orlando sentinel please refer job description view requirements job fl orlando',
 'domestic violence domestic violence traumatic brain injury project coordinator desirable project coordina

In [7]:
cleaned_jobs_collection[0]

'immediate opening immediate opening ar clerk needed foward resumes hrwallsfurnitureaolcom hrwallsfurnitureaolcom web  receivablecash applications clerk neededcandidates must advanced knowledge computers peripheral office devicesfor immediate consideration candidates foward resumes hrwallsfurnitureaolcom source orlando sentinel please refer job description view requirements job fl orlando'

In [8]:
len(cleaned_jobs_collection)

3915

In [9]:
# settings that you use for count vectorizer will go here
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(cleaned_jobs_collection)

# get the first vector out (for the first document)


In [10]:
tfidf_vectorizer_vectors

<3915x38283 sparse matrix of type '<class 'numpy.float64'>'
	with 544139 stored elements in Compressed Sparse Row format>

In [11]:
tfidf_jobs_description = []
for i in range(len(cleaned_jobs_collection)):
    first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[i]
    tfidf = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
    sorted_tfidf = tfidf.sort_values(by=["tfidf"],ascending=False)
    sorted_tfidf = sorted_tfidf.head(10)
    tfidf_list = []
    for i in range(len(sorted_tfidf)):
        if sorted_tfidf.iloc[i]['tfidf'] >= 0.2:
            tfidf_list.append(sorted_tfidf.iloc[i].name)
    tfidf_jobs_description.append(tfidf_list)


In [12]:
def convert_to_strings(jobs):
    strings = " "
    return strings.join(jobs)

job_description = []
for i in range(len(tfidf_jobs_description)):
    job_description.append(convert_to_strings(tfidf_jobs_description[i]))


In [13]:
job_description

['hrwallsfurnitureaolcom foward orlando immediate devicesfor receivablecash',
 'administrative orlando kennethharkerhealthcentralorg assistant trustees sentinel',
 'tallahassee violence fcadv',
 'mainteance houston',
 'supplybracing crodriguezsurefitlabcom rep podiatry coral sunsentinel springs customer service',
 'wwwcalypsorestaurantcom cookline pompano beach',
 'sales mailing inventory administrative assistant',
 'plusposition entrybilingual davenportccaricocom engspanmf mustoracle dataorder ordersverify willprocess wkends requirementsprevious',
 'texcellent service',
 'stevedores mule infosunterminalscom mcintosh checkers container',
 'controllawn eng pest dr tech exp speak nickfelicionejobsbellsouthnet',
 'cdl past driverstrainees yearsno class',
 'oakland park city',
 'urgent md',
 'semiskilled fitters pipe operators south akushnerwhartonsmithcom construction',
 'summit owens maintenance',
 'trainersecurity inst firearms chicago',
 'developmental disabilities mustemail pgibatpers

In [14]:
# for i in range(len(job_description)):
vectorizer = CountVectorizer().fit_transform(job_description)
vectors = vectorizer.toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
vector = np.where(vectors > 0, 1, vectors)

In [16]:
cos_sim_matrix = cosine_similarity(vectors)


In [17]:
cos_sim_matrix[33][18]

0.13363062095621217

In [18]:
def job_id_to_index(job_id):
    return jobs.index[jobs['JobID'] == job_id].values[0]

def job_index_to_id(job_index):
    return jobs.iloc[job_index]['JobID']
    
def find_similar_job(job_id):
    job_index = job_id_to_index(job_id)
    similar_jobs = []
    for i in range(len(vectors)):
        if cos_sim_matrix[job_index][i] >= 0.1 and i != job_index:
            similar_jobs.append(jobs.iloc[i]['JobID'])
    return similar_jobs

In [19]:
find_similar_job(2121)

[908,
 17474,
 152528,
 159411,
 217628,
 246306,
 280275,
 302188,
 302194,
 302282,
 311769,
 331253,
 332608,
 448318,
 464859,
 480396,
 504754,
 522256,
 576968,
 582076,
 584969,
 597000,
 600506,
 608556,
 613873,
 627388,
 630919,
 638922,
 750721,
 821176,
 846109,
 860500,
 860501,
 870732,
 896236,
 899195,
 901582,
 906232,
 912782,
 922397,
 943754,
 1036760,
 1049215,
 1050796,
 1057545,
 1057569,
 1061283,
 1088282,
 1094348,
 1113088]

In [20]:
def cosine_sim(job_id_1, job_id_2):
    job_index_1 = job_id_to_index(job_id_1)
    job_index_2 = job_id_to_index(job_id_2)
    return cos_sim_matrix[job_index_1][job_index_2] 

In [21]:
def user_applied_jobs_check(user_id, job_id):
    user_applied_jobs = apps.loc[apps['UserID'] == user_id]
    if job_id in user_applied_jobs['JobID'] is False:
        return 0
    else:
        return 1

def user_applied_jobs_list(user_id):
    user_applied_jobs = apps.loc[apps['UserID'] == user_id]
    return user_applied_jobs['JobID']

In [22]:
def user_job_pref(user_id, job_id):
    similar_jobs = find_similar_job(job_id)
    user_apply_job = user_applied_jobs_list(user_id)
    mutual_job = set(similar_jobs) & set(user_apply_job)
    if len(mutual_job) == 0:
        return 0
    sum1 = 0 
    sum2 = 0
    for sim_job_id in mutual_job:
        sum1 = cosine_sim(job_id, sim_job_id) * user_applied_jobs_check(user_id, sim_job_id) + sum1
    return sum1
    

In [23]:
user_job_pref(47, 284009)

1.224744871391589

In [33]:
def recommed_jobs(user_id):
    job_id_recommend = pd.DataFrame(columns=['JobID', 'Preference'])
    for i in range(3915):
        job_id = job_index_to_id(i)
        preference = user_job_pref(user_id, job_id)
        if preference > 0.3:
            job_id_recommend = job_id_recommend.append({'JobID': job_id,'Preference':preference}, ignore_index=True)
    job_id_recommendation = job_id_recommend.sort_values(by=['Preference'],ascending=False)
    return job_id_recommendation['JobID']

In [ ]:
accuracy = []
TP = 0 
TN = 0
FN = 0
FP = 0

for i in range(10):
    user_id = users.iloc[i]['UserID']
    print(user_id)
    recommend_list = recommed_jobs(user_id)
    app_list = user_applied_jobs_list(user_id)
    print(set(recommend_list) & set(app_list))
    TP = len(set(recommend_list) & set(app_list)) + TP
    FN = (len(set(app_list)) - len(set(recommend_list) & set(app_list))) + FN
    FP = (len(set(recommend_list)) - len(set(recommend_list) & set(app_list))) + FP
    
precision = TP / (TP + FP)
recall = TP / (TP + FN)

47
{284009, 848187, 733748}
72
{564184, 180313, 480634}
80
set()
98
set()
123


In [31]:
precision

0.043478260869565216

In [32]:
recall

0.2857142857142857

In [ ]:
TP

In [ ]:
FN

In [ ]:
FP